# データを分類してみよう．

## 分類問題
データをクラスに分ける問題を分類問題という．最も簡単な分類問題であるAND計算を人工知能に解かしてみよう．AND計算とは，入力が2つ出力が1つあり，その入力の両方が1のとき出力が1，それ以外のときは0を出力する計算である．AND計算は次の表で表される．$x_1$，$x_2$が入力，$y$が出力である．この入力と出力をプログラムで書いてみよう．

|x1|x2|y |
|--|--|--|
|0 |0 |0 |
|0 |1 |0 |
|1 |0 |0 |
|1 |1 |1 |



In [1]:
import torch

x = torch.tensor(
    [[0.0, 0.0],
     [0.0, 1.0],
     [1.0, 0.0],
     [1.0, 1.0]])

このコードにより入力すべてが，変数xに入れられる．入力は$(x_1, x_2)=(0,0), (0,1), (1,0), (1,1)$の4つのベクトルである．$x$はこの4つのベクトルをまとめた形となっている．人工知能の世界では，これをテンソルと呼ぶ．

入力ベクトルはそれぞれ出力がある．分類問題では出力はラベルと呼ばれる．AND計算ではラベルは0と1である．次にラベルデータを作ってみる．

In [2]:
label = torch.tensor([0, 0, 0, 1])

labelの各要素はそれぞれ，ベクトルに対応している．例えば，ベクトル(0, 0)にはラベル0がついている．分類問題では，このベクトルとラベルの対応を機械に覚えさせてみる．

## 人工ニューラルネットワーク
識別問題を解く方法には様々ある．今回は人工ニューラルネットワークでこの分類問題を解いてみよう．次のコードはニューラルネットワークの構造が書かれたクラスである．


In [3]:
#ニューラルネットワークに必要なライブラリを読み込む
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

#ニューラルネットワークの構造
class NeuralNetwork(nn.Module):
    def __init__(self):
        #ニューラルネットワークの部品を作る
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear( 2, 2) #中間層 (入力の数，出力の数)
        self.fc2 = nn.Linear( 2, 2) #出力層 (入力の数，出力の数)

    def forward(self, input):
        #ニューラルネットワークで行う計算を書く
        h = self.fc1(input) #中間層の計算
        h = F.relu(h)       #活性化関数
        h = self.fc2(h)     #出力層の計算
        h = F.log_softmax(h, dim=1) #出力層の出力の計算
        return h


今回作成するニューラルネットワークは入力層，中間層，出力層の2層で構成する（入力層は数えない）．中間層と出力層は2個のニューロンで構成され，それぞれのニューロンが2つ入力を持つ．

出力は0か1の数値だから出力は1つのニューロンで良さそうに思える．しかし，ニューラルネットワークでは，各ラベルに対し出力のニューロンを1つもたせる．今回は0と1の2種類のラベルがあるため，出力のニューロンは2つある．つまり，AND計算は次の表で表されることになる．

|x1|x2|y1|y2|
|--|--|--|--|
|0 |0 |1 |0 |
|0 |1 |1 |0 |
|1 |0 |1 |0 |
|1 |1 |0 |1 |

ニューロンは下の層から入力を受け取る．今回はニューロンは下の層のすべてのニューロンから入力を受け取るとする．これを全結合という．ニューロンは，下の層からの入力をそのまま受け取るのではなく，入力に重みをかけたものを受け取る．下の層のニューロンの出力を$f(x)$，重みを$w$とすると，ニューロンの前活性$z_1$は次のように書ける．

$z_1=w_11 f(x_1) + w_12 f(x_2)$

つまり，前活性は単純に重みかけ出力の線形和となっている．pytorchでは，Linearという関数を用いることでこの線形和を実現している．

```
self.fc1 = nn.Linear( 2, 2)
```
は，入力層と中間層の間の全結合を表し，
```
self.fc2 = nn.Linear( 2, 2)
```
は，中間層と出力層の間の全結合を表す．便利なことに，ニューロン一個一個全結合を作る必要はなく，nn.Linear( 2, 2)と書くだけで，入力層のニューロン2個，出力層のニューロン2個の全結合を一気に作ることが出来る．

ニューロンの出力は前活性をそのまま出力するのではなく，関数$f()$を通した値が出力される．つまり，前活性を$z$とすると，ニューロンの出力は$f(z)$となる．関数$f()$を活性化関数という．活性化関数には様々な種類があるが，現在多くの場合，rectified linear function (ReLU: Rectified Linear Unit)が用いられる．コードでは
```
h = F.relu(h)
```
と書かれている．ReLUは，入力$h$が0より小さいときは0を出力し，入力$h$が0以上であれば$h$を出力する．

ニューラルネットワークの出力は，表を見ると0か1を出力するように書いてある．しかし，ニューラルネットワークの計算で出てくるのは実数である．そのため，ニューラルネットワークの出力を0と1に変換する必要がある．最も簡単なやり方は，2つの出力を比べ，最大値を取る出力を1，そうでない出力を0に変換する方法である．この方法をデータを分類するときに採用するのだが，この方法は学習には使えない．

ニューラルネットワークの学習とは，ニューラルネットワークの出力が理想の答えに一致するように重みを変えることである．つまり，出力と答えの差が最小になる重みを探すのである．では，出力と答えの差$E$が最小になる重みを探すにはどうするかというと，勾配法という手法を用いる．勾配法は単純で，$w\leftarrow w - \eta \nabla E(w)$を使い重みwを更新するだけである．$\nabla E(w)$は差の勾配（傾き）である．この方法を用いるためには差の微分を計算できなければならない．つまり，ニューラルネットワークに用いる計算は微分可能でなければならないのである．そこで，最大値を取る代わりに，ソフトな最大を取れるソフトマックス関数を用いる．
$z_j = e^{y_j}/\sum e^{y_j}$
コードでは
```
h = F.log_softmax(h, dim=1)
```
と書かれている．これは少し特殊な書き方していて，softmax関数のlogをとっている．logは単調増加関数なので大小関係は変わらない．

次に，このニューラルネットワークを用いてAND計算を実際に学習するコードを示す．
まず，先に宣言したニューラルネットワークのクラスを使い，ニューラルネットワークの実態を作る（オブジェクトを作る）．そのコードが
```
model = NeuralNetwork() 
```
である．

次に，学習方法を指定している．今回は単純な勾配法を用いるのでコードでは次のように書かれている．
```
optimizer = optim.SGD(model.parameters(),
                      lr=0.5, momentum=0.99, nesterov=True) # 確率的勾配法
```

```
epochs = 20 #学習回数
```
で学習回数を指定している．データ全体を1回重みを更新することを1エポックという．20エポックでは，データ全体を使って，20回重みを更新することになる．次のfor文でepochs回繰り返し処理をする命令が書かれている．
```
for epoch in range(epochs):
```

```
  y_outputs = model(X_data) #ニューラルネットワークの出力を計算
```
の部分でニューラルネットワークの出力を計算している．$X_data$にはデータ全体が入っている．つまり$X_data$には4つの入力が入っているのだが，便利なことに，この命令で4つの入力に対する出力を一気に計算できる．

```
  loss = F.nll_loss(y_outputs, y_true) #出力と正解との差を計算．
```
は出力と答えの差を計算している．

他にも色々命令が書いてあるが，今のところは呪文だと思っておこう．

In [4]:
# ネットワークのオブジェクトを生成
model = NeuralNetwork() 
# 最適化手法（学習方法）を設定
optimizer = optim.SGD(model.parameters(),
                      lr=0.5, momentum=0.99, nesterov=True) # 確率的勾配法

epochs = 20 #学習回数
for epoch in range(epochs):
  optimizer.zero_grad()
  X_data = torch.tensor(x) #xをpytorchの形（tensor）に変換
  y_true = torch.tensor(label) #labelをpytorchの形（tensor）に変換

  y_outputs = model(X_data) #ニューラルネットワークの出力を計算
  loss = F.nll_loss(y_outputs, y_true) #出力と正解との差を計算．
  loss.backward()  #逆伝播
  optimizer.step() #学習
  print("学習回数: ", epoch, "誤差: ", loss)

X_data = torch.tensor(x) 
y_outputs = model(X_data)
print(y_outputs)
print(torch.argmax(y_outputs, axis=1))#最大値を持つ配列の要素番号を出力（要素番号=ラベル番号）

学習回数:  0 誤差:  tensor(0.6674, grad_fn=<NllLossBackward0>)
学習回数:  1 誤差:  tensor(0.5389, grad_fn=<NllLossBackward0>)
学習回数:  2 誤差:  tensor(0.5295, grad_fn=<NllLossBackward0>)
学習回数:  3 誤差:  tensor(0.4975, grad_fn=<NllLossBackward0>)
学習回数:  4 誤差:  tensor(0.4049, grad_fn=<NllLossBackward0>)
学習回数:  5 誤差:  tensor(0.3125, grad_fn=<NllLossBackward0>)
学習回数:  6 誤差:  tensor(0.2242, grad_fn=<NllLossBackward0>)
学習回数:  7 誤差:  tensor(0.1481, grad_fn=<NllLossBackward0>)
学習回数:  8 誤差:  tensor(0.0806, grad_fn=<NllLossBackward0>)
学習回数:  9 誤差:  tensor(0.0398, grad_fn=<NllLossBackward0>)
学習回数:  10 誤差:  tensor(0.0175, grad_fn=<NllLossBackward0>)
学習回数:  11 誤差:  tensor(0.0069, grad_fn=<NllLossBackward0>)
学習回数:  12 誤差:  tensor(0.0025, grad_fn=<NllLossBackward0>)
学習回数:  13 誤差:  tensor(0.0008, grad_fn=<NllLossBackward0>)
学習回数:  14 誤差:  tensor(0.0002, grad_fn=<NllLossBackward0>)
学習回数:  15 誤差:  tensor(6.7797e-05, grad_fn=<NllLossBackward0>)
学習回数:  16 誤差:  tensor(1.7464e-05, grad_fn=<NllLossBackward0>)
学習回数:  17 誤差:  t

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
